In [ ]:
# Importing relevant libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import dataframe_image as dfi

%matplotlib inline

In [ ]:
# Loading Relevant Datasets as Pandas Dataframes
# Crunchbase December 4th 2015 

# Companies
companies_df = pd.read_csv("Data/companies.csv")

# Additions
additions_df = pd.read_csv("Data/additions.csv")

# Acquisitions
acquisitions_df = pd.read_csv("Data/acquisitions.csv")

# Investments
investments_df = pd.read_csv("Data/investments.csv")

# Rounds
rounds_df = pd.read_csv("Data/rounds.csv")

In [ ]:
# Inspecting DF shapes

print("Companies shape: ", companies_df.shape)
print("Additions shape: ", additions_df.shape)
print("Acquisitions shape: ", acquisitions_df.shape)
print("Investments shape: ", investments_df.shape)
print("Rounds shape: ", rounds_df.shape)

In [ ]:
# Inspecting columns and general info

print("\033[1m Companies Info: \033[0m") 
companies_df.info()

print("\n\033[1m Additions Info: \033[0m") 
additions_df.info()

print("\n\033[1m Acquisitions Info: \033[0m") 
acquisitions_df.info()

print("\n\033[1m Investments Info: \033[0m") 
investments_df.info()

print("\n\033[1m Rounds Info: \033[0m") 
rounds_df.info()

In [ ]:
# Exporting sample of 5 entries from companies.csv as png to use in report
sample_comp_df = companies_df.sample(n=5)

dfi.export(sample_comp_df,"Sample_companies.png")

In [ ]:
# Inspecing number of NaN values in every column in Companies
companies_df.isnull().sum(axis = 0)

In [ ]:
# Dropping all NaN in "first_funding_at" and "name" columns
cleaned_companies_df = companies_df.dropna(subset=["first_funding_at", "name"])

# Inspecting shape of companies df and NaN columns
print(cleaned_companies_df.shape)
print("\n\033[1mNaN values in every column in Companies after cleaning names and first funding at:\n\033[0m", cleaned_companies_df.isnull().sum(axis = 0))

In [ ]:
# Inspecting range of values in "first_funding_at" in Companies DF
cleaned_companies_df.sort_values("first_funding_at", ascending=True)

# Inspecting range of values in "last_funding_at" in Companies DF
cleaned_companies_df.sort_values("last_funding_at", ascending=False)

# Removing entries where "first_funding_at" and "last_funding_at" is not
# in the 20th or 21st century as this simply dos not make sense
cleaned_companies_df = cleaned_companies_df[cleaned_companies_df.first_funding_at.str.match(r"(19)|(20)")]
cleaned_companies_df = cleaned_companies_df[cleaned_companies_df.last_funding_at.str.match(r"(19)|(20)")]

cleaned_companies_df.shape

In [ ]:
# Converting "first_funding_at" and "last_funding_at" to Pandas Date Time Objects to perform future time operations
cleaned_companies_df["first_funding_at"] = pd.to_datetime(cleaned_companies_df["first_funding_at"])
cleaned_companies_df["last_funding_at"] = pd.to_datetime(cleaned_companies_df["last_funding_at"])

cleaned_companies_df.info()

In [ ]:
# Dropping URL Column as it does not provide much
cleaned_companies_df.drop(axis=1, columns=["homepage_url"], inplace=True)

In [ ]:
# Filter companies to "first_funding_at" to be post 2000
# as this can be seen to be the more modern era of VC and Startups after the dot com crash
cleaned_companies_df = cleaned_companies_df[cleaned_companies_df['first_funding_at'] >= '2000-01-01']
cleaned_companies_df.shape

In [ ]:
# Filling '-' with NaNs to be treated as numeric in "funding_total_usd" column
cleaned_companies_df.funding_total_usd.replace('-', np.nan, inplace=True)

# Converting "funding_total_usd" and "funding_rounds" columns from
# objects to float and integer respeectively to perform operations
cleaned_companies_df.funding_total_usd = pd.to_numeric(cleaned_companies_df.loc[:, "funding_total_usd"])
cleaned_companies_df.funding_rounds = pd.to_numeric(cleaned_companies_df.loc[:, "funding_rounds"])

cleaned_companies_df.info()

In [ ]:
# Inspecting "status" column value counts 
# as this indicates whether the startups are success or not 
cleaned_companies_df.status.value_counts()

In [ ]:
# Setting Pandas to not display max rows
pd.set_option("display.max_rows", None)

# Inspecting which countries are part of this dataset and their value counts
cleaned_companies_df.country_code.value_counts()

In [ ]:
# Creating a subset dataset of only US and European countries
# EEA plus Israel
eu_us_countries = [
    "GBR", "FRA", "DEU", "ISR", "ESP", "NLD",
    "SWE", "IRL", "ITA", "CHE", "DNK", "FIN",
    "BEL", "POL", "NOR", "AUT", "PRT", "BGR",
    "EST", "CZE", "HUN", "LVA", "LTU", "GRC",
    "LUX", "ROM", "SVK", "SVN", "ISL", "CYP",
    "MLT", "HRV", "LIE", "USA"]

cleaned_companies_eu__us_df = cleaned_companies_df[cleaned_companies_df['country_code'].isin(eu_us_countries)]

In [ ]:
cleaned_companies_eu__us_df.info()
print(cleaned_companies_eu__us_df.status.value_counts())
print(cleaned_companies_eu__us_df.funding_rounds.value_counts())

In [ ]:
cleaned_companies_df.funding_rounds.value_counts()

In [ ]:
# Inspecting "status" column value counts after creating EU and US dataset
cleaned_companies_eu__us_df.status.value_counts()

In [ ]:
# Experimenting with removing "operating" companies in "status" column altogether
#cleaned_companies_eu__us_df = cleaned_companies_eu__us_df[np.logical_not(
#    cleaned_companies_eu__us_df.status == 'operating')]

In [ ]:
# Can either only take the non-operating companies to label
# or all non-operating companies plus some operating depending
# on a filter to see them as successful i.e. operating time,
# money raised, number of funding rounds etc.

# Create dataset containing only companies that failed or succeeded
# i.e. companies that are closed, acquired or went through an IPO
# As well as "operating" companies with more than 2 years of operations
#status_companies_df = cleaned_companies_df[cleaned_companies_df['status'].isin(["operating"])]

# Selecting companies with operating status with at least 3 funding rounds as they can be seen as 
# being successful and other opertating companies are excluded as it is too early to tell if they
# are or are not successful
status_companies_operating__eu_us_df = cleaned_companies_eu__us_df[(
    cleaned_companies_eu__us_df["status"] == "operating") & (cleaned_companies_eu__us_df["funding_rounds"] >=3)]

# Selecting all non-operating companies i.e. acquired, ipo, closed
# as they can be seen as directly successful or unsuccessful
status_companies_nonoperating_eu_us_df = cleaned_companies_eu__us_df[~cleaned_companies_eu__us_df['status'].isin(["operating"])]

In [ ]:
# Concatenating the operating with three rounds or more 
# with the non-operating companies
status_companies__eu_us_df = pd.concat([status_companies_operating__eu_us_df, status_companies_nonoperating_eu_us_df])

In [ ]:
print(status_companies__eu_us_df.status.value_counts())
print(status_companies__eu_us_df.funding_rounds.value_counts())
print(status_companies__eu_us_df.isnull().sum(axis = 0))

In [ ]:
status_companies__eu_us_df.shape

In [ ]:
#  Creating binary label column where 0 = closed (unsuccessful) and 
# 1 = aqcquired/IPO/operating with 3 or more rounds of funding (successful) 
status_companies__eu_us_df["label"] = 0

status_companies__eu_us_df.loc[status_companies__eu_us_df.status == "operating", "label"] = 1
status_companies__eu_us_df.loc[status_companies__eu_us_df.status == "acquired", "label"] = 1
status_companies__eu_us_df.loc[status_companies__eu_us_df.status == "ipo", "label"] = 1

In [ ]:
status_companies__eu_us_df.label.value_counts()

In [ ]:
status_companies__eu_us_df.sample(5)

# Cleaning and adding "rounds" dataset

In [ ]:
print(rounds_df.shape)
rounds_df.sample(5)

In [ ]:
rounds_df.funding_round_type.value_counts()

In [ ]:
# Removing "post_ipo_equity" and "post_ipo_debt" rows from "funding_round_type" column
cleaned_rounds_df = rounds_df[np.logical_and(
    rounds_df.funding_round_type != 'post_ipo_equity',
    rounds_df.funding_round_type != 'post_ipo_debt')]

print(cleaned_rounds_df.funding_round_type.value_counts())
print(cleaned_rounds_df.shape)

print("\n", cleaned_rounds_df.info())

In [ ]:
# Converting "funded_at" column to Pandas Date Time Objects to perform future time operations
cleaned_rounds_df.funded_at = pd.to_datetime(cleaned_rounds_df["funded_at"])

In [ ]:
cleaned_rounds_df.info()

# Modifying Features

### Adding "time_between_rounds" & "raised_amount_usd" columns as features to be used for the classifiers

In [ ]:
# Sorting cleaned_rounds_df by company_name and funded_at
cleaned_rounds_df.sort_values(by=["company_name", "funded_at"], ascending=True, inplace=True)

# Creating column for time between rounds
cleaned_rounds_df["time_between_rounds"] = cleaned_rounds_df.groupby("company_name").funded_at.diff()

In [ ]:
# Calculate average time between rounds and average round amount raised in USD
avg_time_round = cleaned_rounds_df.groupby("company_name").agg({
    "time_between_rounds":pd.Series.mean, "raised_amount_usd": "mean"}).rename(
    columns={
        "time_between_rounds": "avg_time_between_rounds",
        "raised_amount_usd": "avg_raised_amount_usd"}).reset_index()

In [ ]:
# Merge into main status_companies_df dataframe
status_companies__eu_us_df = status_companies__eu_us_df.merge(
    avg_time_round, how='left', left_on='name', right_on='company_name')

# Removing "company_name" column as it is redundant
status_companies__eu_us_df = status_companies__eu_us_df.drop("company_name", axis=1)

In [ ]:
# Convert timedelta "avg_time_between_rounds" column to float values for classifiers later
# This can be seen as number of days averaging between each funding round for the startup
status_companies__eu_us_df["avg_time_between_rounds"] = status_companies__eu_us_df.avg_time_between_rounds.dt.days

status_companies__eu_us_df.info()

### Processing "category_list" column to be more intuitive and give the single industry area

In [ ]:
# Moving on to "category_list" column which says the industry(ies) the startup is in

status_companies__eu_us_df.category_list.isna().sum()

# Filling empty categories with "unknown"
status_companies__eu_us_df.category_list.fillna('Unknown', inplace=True)

In [ ]:
# Creating column with list of categories
status_companies__eu_us_df[
    'cat_list'] = status_companies__eu_us_df.category_list.apply(str.split,
                                                                sep='|')

In [ ]:
from collections import Counter

# Creating list of category lists
cats = list(status_companies__eu_us_df.cat_list)

# Flatten list
flat_cats = [cat for sublist in cats for cat in sublist]

# Counting occurences of each
cat_counts = Counter(flat_cats).most_common()

# Distribution
plt.figure(figsize=(12, 7))
plt.bar([x[0] for x in cat_counts[0:25]],
        [x[1] for x in cat_counts[0:25]],
        width=0.8)
plt.xticks(rotation=90)
plt.title('25 Top Industries', size=20)
sns.set_theme()

In [ ]:
# Taking top 25 categories/industries
top_cats = [x[0] for x in cat_counts[0:25]]

# If a company has multiple categories listed, it replaces it with the ones it has in the top 25 only
status_companies__eu_us_df.cat_list = status_companies__eu_us_df.cat_list.map(
    lambda x: list(set(x) & set(top_cats))
    if set(x) & set(top_cats) else ['0_other_cat'])

In [ ]:
# Creating list of category lists
cats = list(status_companies__eu_us_df.cat_list)

# Flatten list
flat_cats = [cat for sublist in cats for cat in sublist]

# Count occurences
cat_counts = Counter(flat_cats).most_common()

# Distribution
plt.figure(figsize=(12, 7))
plt.bar([x[0] for x in cat_counts[0:25]],
        [x[1] for x in cat_counts[0:25]],
        width=0.8)
plt.xticks(rotation=90)
plt.title('Top Industries', size=20)
sns.set_theme()
sns.despine()

In [ ]:
# For future use to create dummy variables in feature matrix
status_companies__eu_us_df.cat_list = status_companies__eu_us_df.cat_list.apply(
    lambda x: x[0])

In [ ]:
# Removing category_list

status_companies__eu_us_df.drop(["category_list"], axis=1, inplace=True)

# Renaming cat_list to Industry
status_companies__eu_us_df.rename(columns={"cat_list": "industry"}, inplace=True)

In [ ]:
# Replacing blank space with underscore in "industry"
status_companies__eu_us_df.industry.replace(" ", "_", regex=True, inplace=True)

In [ ]:
status_companies__eu_us_df.sample(5)

In [ ]:
status_companies__eu_us_df.isnull().sum(axis = 0)

In [ ]:
status_companies__eu_us_df.info()

# Adding Investor dataset

In [ ]:
investments_df.info()

In [ ]:
# Counting number of investors per startup
investor_count = investments_df[["company_permalink",
                         "investor_permalink"]].groupby(["company_permalink"]).agg(["count"])

In [ ]:
# Merging DFs
status_companies__eu_us_df = pd.merge(how="left",left=status_companies__eu_us_df, right=investor_count, 
                                 left_on="permalink", right_on="company_permalink")

In [ ]:
# Renaming "investor_permalink" to num_of_investors
status_companies__eu_us_df.rename(columns={status_companies__eu_us_df.columns[-1]: "num_of_investors"}, inplace=True)

In [ ]:
status_companies__eu_us_df.info()
#status_companies_df.isnull().sum(axis = 0)

In [ ]:
# Dropping permalink Column as they do not provide much
status_companies__eu_us_df.drop(axis=1, columns=["permalink"], inplace=True)

In [ ]:
# Changing NaN values to "unknown"  for "country_code"
#status_companies__eu_us_df.country_code.fillna('Unknown', inplace=True)

status_companies__eu_us_df.isnull().sum(axis = 0)

In [ ]:
status_companies__eu_us_df.info()


# Countries Distribution

country_dist = status_companies__eu_us_df.groupby(
    'country_code').size().sort_values(ascending=False)
plt.bar(country_dist[0:20].index, height=country_dist[0:20].values)
plt.xticks(rotation=90)
sns.despine()
plt.title('Distribution of Top 10 Countries')

# Outlier Detection and Removal

In [ ]:
status_companies__eu_us_df.describe()

In [ ]:
# Boxplots to see skewness

plt.subplots(1, 4, figsize=(15, 5))

plt.subplot(1, 4, 1)
plt.boxplot(status_companies__eu_us_df.avg_time_between_rounds[status_companies__eu_us_df.avg_time_between_rounds.notnull()])
plt.title("Average Time Between Rounds", size=12)

plt.subplot(1, 4, 2)
plt.boxplot(status_companies__eu_us_df.avg_raised_amount_usd[status_companies__eu_us_df.avg_raised_amount_usd.notnull()])
plt.title("Average Raised Amount USD", size=10)

plt.subplot(1, 4, 3)
plt.boxplot(status_companies__eu_us_df.num_of_investors[status_companies__eu_us_df.num_of_investors.notnull()])
plt.title("Number of Investors", size=16)

plt.subplot(1, 4, 4)
plt.boxplot(status_companies__eu_us_df.funding_total_usd[status_companies__eu_us_df.funding_total_usd.notnull()])
plt.title("Total Funding USD", size=16)
plt.show()

In [ ]:
pd.options.display.float_format = '{:20,.2f}'.format
status_companies__eu_us_df["avg_raised_amount_usd"].nlargest(5)

In [ ]:
status_companies__eu_us_df.iloc[9000]

In [ ]:
# Trying to remove NaN
#status_companies__eu_us_df.dropna(inplace=True)
status_companies__eu_us_df.info()

In [ ]:
def locateOutliers(df, column, lim_scalar=1.5):
    """
    Returns outliers above the max limit for a column in a dataframe

    input: DataFrame, column(series),lim_scalar(float)
    output: DataFrame
    """
    q25, q50, q75 = df[column].quantile(q=[0.25, 0.5, 0.75])
    iqr = q75 - q25
    # max limits to be considered an outlier
    max = q75 + lim_scalar * iqr
    # identify the points
    outlier_mask = [True if x > max else False for x in df[column]]
    print(
        "{} outliers found out of {} data points, {:.2f}% of the data".format(
            sum(outlier_mask), len(df[column]),
            100 * (sum(outlier_mask) / len(df[column]))))
    return outlier_mask

In [ ]:
print("Funding total USD: ")
total_funding_outlier_mask = pd.Series(
    locateOutliers(status_companies__eu_us_df, "funding_total_usd", lim_scalar=3))

print("\nAverage Raised Amount USD: ")
avg_raise_outlier_mask = pd.Series(
    locateOutliers(status_companies__eu_us_df, "avg_raised_amount_usd", lim_scalar=3))

print("\nTime between first rounds: ")
first_time_outlier_mask = pd.Series(
    locateOutliers(status_companies__eu_us_df, 'avg_time_between_rounds'))

print("\nNumber of Investors: ")
num_of_investors_outlier_mask = pd.Series(
    locateOutliers(status_companies__eu_us_df, "num_of_investors"))

In [ ]:
# Outlier Removal
clean_df1 = status_companies__eu_us_df[~total_funding_outlier_mask]
clean_df2 = status_companies__eu_us_df[~avg_raise_outlier_mask]
clean_df3 = status_companies__eu_us_df[~first_time_outlier_mask]
clean_df4 = status_companies__eu_us_df[~num_of_investors_outlier_mask]

clean_df = clean_df1.merge(clean_df2,
                           left_on=list(status_companies__eu_us_df.columns),
                           right_on=list(status_companies__eu_us_df.columns),
                           how='inner')

clean_df = clean_df.merge(clean_df3,
                          left_on=list(status_companies__eu_us_df.columns),
                          right_on=list(status_companies__eu_us_df.columns),
                          how='inner')

clean_df = clean_df.merge(clean_df4,
                          left_on=list(status_companies__eu_us_df.columns),
                          right_on=list(status_companies__eu_us_df.columns),
                          how='inner')

In [ ]:
print('Removed {} outliers, {:.2f}% of the original dataset'.format(
    (status_companies__eu_us_df.shape[0] - clean_df.shape[0]), 100 *
    ((status_companies__eu_us_df.shape[0] - clean_df.shape[0]) / 
    (status_companies__eu_us_df.shape[0] + clean_df.shape[0]))))

In [ ]:
# Boxplots to see skewness

plt.subplots(1, 4, figsize=(15, 5))

plt.subplot(1, 4, 1)
plt.boxplot(clean_df.avg_time_between_rounds[clean_df.avg_time_between_rounds.notnull()])
plt.title("Average Time Between Rounds", size=12)

plt.subplot(1, 4, 2)
plt.boxplot(clean_df.avg_raised_amount_usd[clean_df.avg_raised_amount_usd.notnull()])
plt.title("Average Raised Amount USD", size=10)

plt.subplot(1, 4, 3)
plt.boxplot(clean_df.num_of_investors[clean_df.num_of_investors.notnull()])
plt.title("Number of Investors", size=16)

plt.subplot(1, 4, 4)
plt.boxplot(clean_df.funding_total_usd[clean_df.funding_total_usd.notnull()])
plt.title("Total Funding USD", size=16)
plt.show()

In [ ]:
# Seperating between EU and US startups
clean_eu = clean_df[~clean_df["country_code"].isin(["USA"])]

clean_us = clean_df[clean_df["country_code"].isin(["USA"])]

In [ ]:
clean_eu.sample(5)

In [ ]:
clean_us.sample(5)

In [ ]:
clean_eu.info()

In [ ]:
clean_us.info()

In [ ]:
print("Value counts of EU labels:\n ", clean_eu.label.value_counts())
print("\nValue counts of US labels:\n", clean_us.label.value_counts())

In [ ]:
# Filling "avg_time_between_rounds" NaN values with 0 as these 
# have only had 1 round of funding and therefor no time between rounds
clean_eu[["avg_time_between_rounds"]] = clean_eu[["avg_time_between_rounds"]].fillna(value=0)
clean_us[["avg_time_between_rounds"]] = clean_us[["avg_time_between_rounds"]].fillna(value=0)

In [ ]:
#clean_us.sample(10)

# Dropping irrelevant columns for eu dataset:
# "name", "state_code", "region"
clean_eu = clean_eu.drop(columns=["name", "state_code", "region", "founded_at"])

In [ ]:
# Dropping irrelevant columns for us dataset:
# "name", "country_code", "city"
clean_us = clean_us.drop(columns=["name", "country_code", "city", "founded_at"])

In [ ]:
# Dropping NaN "region" for US dataset as there are only 20 entries
# and "funding_total_usd", "avg_raised_amount_usd" ,"num_of_investors"
clean_us = clean_us.dropna(subset=["region", "funding_total_usd", "avg_raised_amount_usd", "num_of_investors"])

In [ ]:
#clean_eu.info()
#clean_eu.isna().sum()
#clean_eu[clean_eu['city'].isnull()]

# Filling NaN "cities" with the most frequent city of the matching "country_code"
#clean_eu["city"] = clean_eu.set_index('country_code', append=True) \
#                    .groupby(level="country_code").city.apply(lambda x: x.fillna(x.value_counts().idxmax())) \
#                    .reset_index('country_code', drop=True)

# Dropping NaN "city" ,"avg_raised_amount_usd", "num_of_investors" for EU dataset
clean_eu = clean_eu.dropna(subset=["city", "avg_raised_amount_usd", "num_of_investors", "funding_total_usd"])

In [ ]:
clean_eu.isna().sum()
#clean_eu.label.value_counts()
#clean_us.info()

In [ ]:
clean_eu.sample(5)

In [ ]:
clean_eu.label.value_counts()

In [ ]:
clean_us.label.value_counts()

In [ ]:
clean_eu.info()

In [ ]:
#Saving clean EU and US to csv in data folder
clean_eu.to_csv('data/clean_eu.csv')
clean_us.to_csv('data/clean_us.csv')